In [1]:
require 'nn'
require 'hdf5'

In [2]:
local Squeeze, parent = torch.class('nn.Squeeze', 'nn.Module')

function Squeeze:updateOutput(input)
    self.size = input:size()
    self.output = input:squeeze()
    return self.output
end

function Squeeze:updateGradInput(input, gradOutput)
  self.gradInput = gradOutput:view(self.size)
  return self.gradInput  
end

In [3]:
myFile = hdf5.open('PTB.hdf5','r')
data = myFile:all()
myFile:close()

In [8]:
word_embed = torch.Tensor(100002,50)
word_embed:copy(data['word_embeddings']:narrow(1,1,100002))

In [9]:
train_input_word_windows = data['train_input_word_windows']
train_input_cap_windows = data['train_input_cap_windows']

train = train_input_word_windows:clone()
train_cap = train_input_cap_windows:clone()
train_output = data['train_output']

In [10]:
dim_hidden = 50
dim_hidden2 = 50
--Define the module
neuralnet = nn.Sequential()

--Include the lookup tables
neuralnet:add(nn.LookupTable(data['nwords'][1],dim_hidden))
neuralnet:add(nn.View(1,-1,5*50))
neuralnet:add(nn.Squeeze()) -- this layer is to go from a 1xAxB tensor to AxB dimensional tensor (https://groups.google.com/forum/#!topic/torch7/u4OEc0GB74k)
neuralnet:add(nn.Linear(5*dim_hidden,dim_hidden2))
neuralnet:add(nn.HardTanh())
neuralnet:add(nn.Linear(dim_hidden2, data['nclasses'][1]))
neuralnet:add(nn.LogSoftMax())

In [6]:
dataset={};
for i=1,train:size(1) do 
  dataset[i] = {train[i]:view(1,5), train_output[i]}
end
function dataset:size() return train:size(1) end -- 100 examples

In [7]:
criterion = nn.ClassNLLCriterion()

In [11]:
trainer = nn.StochasticGradient(neuralnet, criterion)
trainer.learningRate = 0.01
trainer.maxIteration = 10
trainer:train(dataset)

# StochasticGradient: training	


# current error = 0.76410843150984	


# current error = 0.49077146420375	


# current error = 0.39363261704525	


# current error = 0.33967493665589	


...

# current error = 0.2785490815371	


# current error = 0.25929846572195	


# current error = 0.24431658769046	


# current error = 0.23182629793747	


# current error = 0.22073000924219	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 0.22073000924219	


In [35]:
val_word = data['valid_input_word_windows']:clone()
val_output = data['valid_output']:clone()

pred_train = neuralnet:forward(train)
max,argmax_train = pred_train:max(2)
acc_train = 0
for i = 1, train_output:size(1) do
    if argmax_train[i][1] == train_output[i] then
        acc_train = acc_train + 1
    end
end
score_train = acc_train/train_output:size(1)
print('Validation Score on Train is '..score_train)

Validation Score on Train is 0.93472968205236	


In [36]:
pred_val = neuralnet:forward(val_word)
max,argmax_val = pred_val:max(2)
acc_val = 0
for i = 1, data['valid_output']:size(1) do
    if argmax_val[i][1] == data['valid_output'][i] then
        acc_val = acc_val + 1
    end
end
score_val = acc_val/data['valid_output']:size(1)
print('Validation Score on Train is '..score_val)

Validation Score on Train is 0.91536173828599	


## Adding the cap:

In [10]:
train_new = train_input_word_windows:clone()
train_cap_new = train_cap:clone()
train_cap_new:add(100002)

In [11]:
train_new = torch.cat(train_new,train_cap_new,2)

In [37]:
dim_hidden = 50
dim_hidden2 = 50
--Define the module
neuralnet_wc = nn.Sequential()

--Include the lookup tables
neuralnet_wc:add(nn.LookupTable(data['nwords'][1]+4,dim_hidden))
neuralnet_wc:add(nn.View(1,-1,10*dim_hidden))
neuralnet_wc:add(nn.Squeeze()) -- this layer is to go from a 1xAxB tensor to AxB dimensional tensor (https://groups.google.com/forum/#!topic/torch7/u4OEc0GB74k)
neuralnet_wc:add(nn.Linear(10*dim_hidden,dim_hidden2))
neuralnet_wc:add(nn.HardTanh())
neuralnet_wc:add(nn.Linear(dim_hidden2, data['nclasses'][1]))
neuralnet_wc:add(nn.LogSoftMax())

In [23]:
dataset2={};
for i=1,train:size(1) do 
  dataset2[i] = {train_new[i]:view(1,10), train_output[i]}
end
function dataset2:size() return train:size(1) end -- 100 examples

In [38]:
trainer2 = nn.StochasticGradient(neuralnet_wc, criterion)
trainer2.learningRate = 0.01
trainer2.maxIteration = 10
trainer2:train(dataset2)

# StochasticGradient: training	


# current error = 0.64715155130193	


# current error = 0.4320182739747	


# current error = 0.35041711557732	


# current error = 0.30323407053154	


...

# current error = 0.24901295649474	


# current error = 0.23269701198418	


# current error = 0.21847914232537	


# current error = 0.20689133202205	


# current error = 0.19687390394816	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 0.19687390394816	


In [39]:
pred_train2 = neuralnet_wc:forward(train_new)
max,argmax_train2 = pred_train2:max(2)
acc_train2 = 0
for i = 1, train_output:size(1) do
    if argmax_train2[i][1] == train_output[i] then
        acc_train2 = acc_train2 + 1
    end
end
score_train2 = acc_train2/train_output:size(1)
print('Validation Score on Train is '..score_train2)


Validation Score on Train is 0.94118658961767	


In [30]:
valid_new = data['valid_input_word_windows']:clone()
valid_cap_new = data['valid_input_cap_windows']:clone()
valid_cap_new:add(100002)
valid_new = torch.cat(valid_new,valid_cap_new,2)

In [40]:
pred_val2 = neuralnet_wc:forward(valid_new)
max,argmax_val2 = pred_val2:max(2)
acc_val2 = 0
for i = 1, data['valid_output']:size(1) do
    if argmax_val2[i][1] == data['valid_output'][i] then
        acc_val2 = acc_val2 + 1
    end
end
score_val2 = acc_val2/data['valid_output']:size(1)
print('Validation Score on Train is '..score_val2)

Validation Score on Train is 0.92946558630736	


#### New test: 

In [41]:
dim_hidden = 100
dim_hidden2 = 60
--Define the module
neuralnet_wc2 = nn.Sequential()

--Include the lookup tables
neuralnet_wc2:add(nn.LookupTable(data['nwords'][1]+4,dim_hidden))
neuralnet_wc2:add(nn.View(1,-1,10*dim_hidden))
neuralnet_wc2:add(nn.Squeeze()) -- this layer is to go from a 1xAxB tensor to AxB dimensional tensor (https://groups.google.com/forum/#!topic/torch7/u4OEc0GB74k)
neuralnet_wc2:add(nn.Linear(10*dim_hidden,dim_hidden2))
neuralnet_wc2:add(nn.HardTanh())
neuralnet_wc2:add(nn.Linear(dim_hidden2, data['nclasses'][1]))
neuralnet_wc2:add(nn.LogSoftMax())

In [42]:
trainer3 = nn.StochasticGradient(neuralnet_wc2, criterion)
trainer3.learningRate = 0.01
trainer3.maxIteration = 15
trainer3:train(dataset2)

# StochasticGradient: training	


# current error = 0.62711629285765	


# current error = 0.43568069541595	


# current error = 0.36333161950036	


# current error = 0.31800302668997	


...

# current error = 0.19445889801514	


# current error = 0.18557630721415	


# current error = 0.17792761519416	


# current error = 0.17149989155073	


# current error = 0.16545346801062	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 0.16545346801062	


In [43]:
pred_train3 = neuralnet_wc2:forward(train_new)
max,argmax_train3 = pred_train3:max(2)
acc_train3 = 0
for i = 1, train_output:size(1) do
    if argmax_train3[i][1] == train_output[i] then
        acc_train3 = acc_train3 + 1
    end
end
score_train3 = acc_train3/train_output:size(1)
print('Validation Score on Train is '..score_train3)

Validation Score on Train is 0.94907337404921	


In [45]:
pred_val3 = neuralnet_wc2:forward(valid_new)
max,argmax_val3 = pred_val3:max(2)
acc_val3 = 0
for i = 1, data['valid_output']:size(1) do
    if argmax_val3[i][1] == data['valid_output'][i] then
        acc_val3 = acc_val3 + 1
    end
end
score_val3 = acc_val3/data['valid_output']:size(1)
print('Validation Score on Validation is '..score_val3)

Validation Score on Validation is 0.93164299587278	


### Saving the models:

In [46]:
torch.save('nn1',neuralnet_wc)

In [47]:
torch.save('nn2',neuralnet_wc2)

### Testing on the last model:

In [48]:
data

{


  train_output : LongTensor - size: 912666


  train_input_cap_windows : LongTensor - size: 912666x5


  test_input_word_windows : LongTensor - size: 129696x5


  valid_output : LongTensor - size: 131808


...

  test_input_cap_windows : LongTensor - size: 129696x5


  train_input_word_windows : LongTensor - size: 912666x5


  nclasses : IntTensor - size: 1


  word_embeddings : DoubleTensor - size: 400002x50


  valid_input_word_windows : LongTensor - size: 131808x5
}


In [49]:
test_words = data['test_input_word_windows']:clone()
test_cap = data['test_input_cap_windows']:clone()
test_cap:add(100002)
test_data = torch.cat(test_words,test_cap,2)

In [51]:
output_test = neuralnet_wc2:forward(test_data)
max,pred_test_nn2 = output_test:max(2)

In [52]:
pred_test_nn2:size()

 129696
      1
[torch.LongStorage of size 2]



In [53]:
myFile = hdf5.open('test_nn2.h5', 'w')
myFile:write('dataset1', pred_test_nn2)
myFile:close()

### Word embedings

In [21]:
dim_hidden = 50
dim_hidden2 = 50
--Define the module
neuralnet_wc3 = nn.Sequential()

LT = nn.LookupTable(data['nwords'][1]+4,dim_hidden)
LT.weight:narrow(1,1,data['nwords'][1]):copy(word_embed)
--Include the lookup tables
neuralnet_wc3:add(LT)
neuralnet_wc3:add(nn.View(1,-1,10*dim_hidden))
neuralnet_wc3:add(nn.Squeeze()) -- this layer is to go from a 1xAxB tensor to AxB dimensional tensor (https://groups.google.com/forum/#!topic/torch7/u4OEc0GB74k)
neuralnet_wc3:add(nn.Linear(10*dim_hidden,dim_hidden2))
neuralnet_wc3:add(nn.HardTanh())
neuralnet_wc3:add(nn.Linear(dim_hidden2, data['nclasses'][1]))
neuralnet_wc3:add(nn.LogSoftMax())

In [22]:
criterion = nn.ClassNLLCriterion()

In [24]:
trainer4 = nn.StochasticGradient(neuralnet_wc3, criterion)
trainer4.learningRate = 0.01
trainer4.maxIteration = 15
trainer4:train(dataset2)

# StochasticGradient: training	


# current error = 0.30321519416855	


# current error = 0.16526912050898	


# current error = 0.13882272715113	


# current error = 0.12316241705001	


...

# current error = 0.082638819089707	


# current error = 0.080688000292496	


# current error = 0.077611036617596	


# current error = 0.074708496370951	


# current error = 0.072265615799489	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 0.072265615799489	


In [25]:
pred_train4 = neuralnet_wc3:forward(train_new)
max,argmax_train4 = pred_train4:max(2)
acc_train4 = 0
for i = 1, train_output:size(1) do
    if argmax_train4[i][1] == train_output[i] then
        acc_train4 = acc_train4 + 1
    end
end
score_train4 = acc_train4/train_output:size(1)
print('Validation Score on Train is '..score_train4)

Validation Score on Train is 0.97479472227518	


In [27]:
valid_new = data['valid_input_word_windows']:clone()
valid_cap_new = data['valid_input_cap_windows']:clone()
valid_cap_new:add(100002)
valid_new = torch.cat(valid_new,valid_cap_new,2)

In [28]:
pred_val4 = neuralnet_wc3:forward(valid_new)
max,argmax_val4 = pred_val4:max(2)
acc_val4 = 0
for i = 1, data['valid_output']:size(1) do
    if argmax_val4[i][1] == data['valid_output'][i] then
        acc_val4 = acc_val4 + 1
    end
end
score_val4 = acc_val4/data['valid_output']:size(1)
print('Validation Score on Validation is '..score_val4)

Validation Score on Validation is 0.95920581451809	


In [29]:
test_words = data['test_input_word_windows']:clone()
test_cap = data['test_input_cap_windows']:clone()
test_cap:add(100002)
test_data = torch.cat(test_words,test_cap,2)

In [32]:
output_test = neuralnet_wc3:forward(test_data)
max,pred_test_nn3 = output_test:max(2)

In [33]:
myFile = hdf5.open('test_nn3.h5', 'w')
myFile:write('dataset1', pred_test_nn3)
myFile:close()